In [ ]:
! conda install -c conda-forge ipywidgets

In [1]:
import torch
import os
import pandas as pd
from torch_geometric.data import InMemoryDataset, Data, download_url, extract_zip
from torch_geometric.utils.convert import to_networkx
import networkx as nx
from tqdm import tqdm
import matplotlib.pyplot as plt

C:\Users\qsavy_1\.conda\envs\torch_py\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: [WinError 126] The specified module could not be found

In [2]:
#load datasets
channel = pd.read_csv('channel_data_final.csv')
video = pd.read_csv('video_data_final.csv')


# rename channelTitle to channelName
video = video.rename({'channelTitle' : 'channelName'}, axis=1)

# drop deprecated columns
channel.drop('Unnamed: 0', inplace=True, axis=1)
video.drop('Unnamed: 0', inplace=True, axis=1)

# create numerical id for all channels
channel['channelId'] = pd.factorize(channel.channelName)[0]
video['channelId'] = pd.factorize(video.channelName)[0]

In [3]:
channel.head()

,channelName,subscribers,views,totalVideos,playlistId,channelId
0,EEG,1740000,1616271673,15332,UUZSJ9bCN0Kh8vJImfacvkFg,0
1,Teleantioquia,511000,121905317,38510,UUeq_SgWZDF8h-ujzwBXI5jQ,1
2,Juan De Montreal,1650000,479065947,1075,UUtUIhr_2EOuLdeAnFYMNnkg,2
3,Katia Nabil,1260000,127790680,391,UUyIE18PSCKxomVDaJ0lEGFw,3
4,Exitosa Noticias,749000,284204440,39447,UUxgO_rak_BKZP8VNVmYqbWg,4


In [4]:
video.head(50)

,video_id,channelName,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,...,definition,caption,pushblishDayName,durationSecs,tagsCount,likeRatio,commentRatio,titleLength,title_no_stopwords,channelId
0,uCqLtZhf8xw,ElTrollino,¡PASAMOS de TIENDA de MASCOTAS NOOB a TIENDA d...,Vídeo en español de Roblox Tycoon donde pasamo...,['roblox'],2022-07-28T18:15:03Z,819935.0,35640.0,NaN,1343.0,...,hd,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,xkQKH6-uNyE,ElTrollino,¡HACKER NOS ENCIERRA EN LA PRISIÓN DE BEDROCK!...,El hacker nos encierra en la prisión más segur...,['minecraft'],2022-06-18T20:45:18Z,3900502.0,112102.0,NaN,2620.0,...,hd,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,MWNpBHATT-w,ElTrollino,¡NO ENCUENTRO a LYNA en el LABERINTO! 😱 JUGAMO...,Vlog de Paris y Disney Land donde jugamos al e...,['escondite'],2022-06-10T18:54:53Z,2827316.0,77892.0,NaN,1710.0,...,hd,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,6Qg_FpyPY-Y,ElTrollino,¡PASAMOS de GRANJA NOOB a GRANJA PRO en ROBLOX...,Vídeo en español de Farm life Tycoon donde pas...,['roblox'],2022-06-04T18:20:42Z,2692418.0,80655.0,NaN,2659.0,...,hd,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,IgNj4zJ-Tv0,ElTrollino,¡Me ESCONDO DENTRO de una MESA DE CRAFTEO! 😂 E...,"El escondite en minecraft con mods, me escondo...",['minecraft'],2022-05-28T18:00:16Z,4171162.0,196737.0,NaN,3183.0,...,hd,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
5,nw8BIIQeQAo,ElTrollino,¡NADIE ENCUENTRA A TROLLINO ESCONDIDO EN EL CU...,¡Rec Room es gratis y está disponible en todas...,['escondite'],2022-04-30T18:00:22Z,4007187.0,101977.0,NaN,2208.0,...,hd,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
6,qy6AJwQV96w,ElTrollino,Nos ponemos GORDITOS en la CENA más cara de LO...,Los Compas @ElTrollino @Mikecrack @Timba Vk @A...,"['los compas', 'eltrollino', 'mikecrack']",2022-04-29T18:00:13Z,2521662.0,109711.0,NaN,3730.0,...,hd,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7,vaiAW3HvCEY,ElTrollino,¡Uso cámaras de seguridad para hacer TRAMPAS e...,¡Uso cámaras de seguridad para hacer TRAMPAS e...,['minecraft'],2022-04-14T18:00:13Z,7614516.0,279360.0,NaN,5074.0,...,hd,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
8,T35m_FiXdQs,ElTrollino,¡SOBREVIVE a la MEGA RAMPA TROLL en GTA 5! 😱 C...,Carrera de cara a cara en la mega rampa troll ...,['gta 5'],2022-03-28T19:26:14Z,9307456.0,190487.0,NaN,2958.0,...,hd,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9,P7OY6yXHaB4,ElTrollino,MIKELLINO GIGANTE vs LOS COMPAS ENANOS 😂 Roblo...,Hoy hemos jugado un mapa de parkour complicado...,['roblox'],2022-03-18T19:57:57Z,9968384.0,200598.0,NaN,3846.0,...,hd,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [9]:
!pip install -U sentence-transformers

  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Using cached transformers-4.21.3-py3-none-any.whl (4.7 MB)
  Using cached torchvision-0.13.1-cp37-cp37m-win_amd64.whl (1.1 MB)
  Using cached nltk-3.7-py3-none-any.whl (1.5 MB)
  Using cached sentencepiece-0.1.97-cp37-cp37m-win_amd64.whl (1.1 MB)
  Using cached huggingface_hub-0.9.1-py3-none-any.whl (120 kB)
  Using cached filelock-3.8.0-py3-none-any.whl (10 kB)
  Using cached tokenizers-0.12.1-cp37-cp37m-win_amd64.whl (3.3 MB)
  Using cached regex-2022.9.13-cp37-cp37m-win_amd64.whl (267 kB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125940 sha256=81768d3f1760a08c91dcc3e4518699174bc5b10b5543cca5546ae947be4c0cf9
  Stored in directory: c:\users\qsavy_1\appdata\local\pip\cache\wheels\bf\06\fb\d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: torch
    Foun

In [10]:
from sentence_transformers import SentenceTransformer

In [ ]:
def load_node_csv(path, index_col, encoders=None, **kwargs):
    df = video
    mapping = {index: i for i, index in enumerate(df.index.unique())}

    x = None
    if encoders is not None:
        xs = [encoder(df[col]) for col, encoder in encoders.items()]
        x = torch.cat(xs, dim=-1)

    return x, mapping

In [ ]:
class SequenceEncoder(object):
    def __init__(self, model_name='all-MiniLM-L6-v2', device=None):
        self.device = device
        self.model = SentenceTransformer(model_name, device=device)

    @torch.no_grad()
    def __call__(self, df):
        x = self.model.encode(df.values, show_progress_bar=True,
                              convert_to_tensor=True, device=self.device)
        return x.cpu()

In [ ]:
#cambiar features para encoder
# class videoEncoder(object):
#     def __init__(self, sep='|'):
#         self.sep = sep

#     def __call__(self, df):
#         video = set(g for col in df.values for g in col.split(self.sep))
#         mapping = {video: i for i, video in enumerate(video)}

#         x = torch.zeros(len(df), len(mapping))
#         for i, col in enumerate(df.values):
#             for video in col.split(self.sep):
#                 x[i, mapping[video]] = 1
#         return x

In [ ]:
class tagEncoder(object):
    def __init__(self, sep=','):
        self.sep = sep

    def __call__(self, df):
        tags = set(g for col in df.values for g in col.split(self.sep))
        mapping = {tag: i for i, tag in enumerate(tags)}

        x = torch.zeros(len(df), len(mapping))
        for i, col in enumerate(df.values):
            for tag in col.split(self.sep):
                x[i, mapping[tag]] = 1
        return x

In [ ]:
video_x, video_mapping = load_node_csv(
    video, index_col='channelId', encoders={
        'channelName': SequenceEncoder(),
        'tags': tagEncoder()
    })

In [ ]:
video.head()

In [ ]:
_, channel_mapping = load_node_csv(channel, index_col='channelId')

In [ ]:
from torch_geometric.data import HeteroData

data = HeteroData()

data['videos'].num_nodes = len(video_mapping)  # content creators do not have any features.
data['channelName'].x = video_x

print(data)


In [ ]:
def load_edge_csv(path, src_index_col, src_mapping, dst_index_col, dst_mapping,
                  encoders=None, **kwargs):
    df = video

    src = [src_mapping[index] for index in df[src_index_col]]
    dst = [dst_mapping[index] for index in df[dst_index_col]]
    edge_index = torch.tensor([src, dst])

    edge_attr = None
    if encoders is not None:
        edge_attrs = [encoder(df[col]) for col, encoder in encoders.items()]
        edge_attr = torch.cat(edge_attrs, dim=-1)

    return edge_index, edge_attr

In [ ]:
class IdentityEncoder(object):
    def __init__(self, dtype=None):
        self.dtype = dtype

    def __call__(self, df):
        return torch.from_numpy(df.values).view(-1, 1).to(self.dtype)

In [ ]:
edge_index, edge_label = load_edge_csv(
    channel,
    src_index_col='channelName',
    src_mapping=channel_mapping,
    dst_index_col='totalVideos',
    dst_mapping=video_mapping,
    encoders={'totalVideos': IdentityEncoder(dtype=torch.long)},
)

data['channelId', 'viewCount', 'totalVideos'].edge_index = edge_index
data['channelId', 'viewCount', 'totalVideos'].edge_label = edge_label

print(data)